### Phase 1: 

Develop a function/algorithm to generate metrics to gauge the impact of the dendritic extension proofreading task on the connectome. Specific metrics include determining the number of additional synapses that are added to the connectome and the volume of the mesh that is added to the connectome.


In [2]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
from data_loader import get_syn_counts
from cloudvolume import CloudVolume
import trimesh

In [3]:
client=CAVEclient()
vol = CAVEclient('minnie65_phase3_v1')

In [4]:
df = pd.read_csv('expanded_gt.csv')
len(set(df['Root_id'].to_list()))

15

In [5]:
#root_id = rids[ind]
datastack_name = "minnie65_phase3_v1"
client = CAVEclient(datastack_name)
vol = CloudVolume(
        client.info.segmentation_source(),
        use_https=True,
        progress=False,
        bounded=False,
        fill_missing=True,
        secrets={"token": client.auth.token}
    )
mesh = vol.mesh.get(str(864691135526309723))[864691135526309723]

mesh_obj = trimesh.Trimesh(mesh.vertices, mesh.faces)

mesh_obj.volume

1414939409226.0

In [11]:
def get_em(x_pre, x_post, y_pre, y_post, z_pre, z_post):
    em = CloudVolume("s3://bossdb-open-data/iarpa_microns/minnie/minnie65/em", use_https=True, mip=0)
    return em[x_pre:x_post, y_pre:y_post, z_pre:z_post]

In [9]:
# function to compute metrics for the first extension


def metrics(root_id, extensions):
    '''
    Parameters:
        root_id: (str) original segmentation root id
        extensions: (list object) all segmentation extension candidates

    Return:
        total_synapses: total number of synapses added
        volume: total mesh volume added
        total_run_len : total run length of correct extension segid added
    '''

    # get correct seg_id from ground truth table
    ground_truth = pd.read_csv('expanded_gt.csv')

    # from ground truth table: extract the correct segid candidate

    correct_extension = df.loc[df['Root_id'] == (str(864691136577830164) + '/')]

    # drop null extensions
    correct_extension.drop(correct_extension.index[correct_extension['Ext'] == 'null/'], inplace = True)

    all_extensions = correct_extension['Ext'].to_list()

    total_synapses = 0

    # confirm later --> mesh_total (numerical quantity - float)
    mesh_total = 0

    for i in all_extensions:
        pre_ct, post_ct = get_syn_counts(i[:-1])
        total_synapses += pre_ct + post_ct
        mesh = vol.mesh.get(str(i))[i]
        mesh_obj = trimesh.Trimesh(mesh.vertices, mesh.faces)
        mesh_total += (mesh_obj.volume)

    incorrect_synapses = 0
    mesh_total_inc = 0
    for k in extensions:
        if k not in all_extensions:
            pre_ct_k, post_ct_k = get_syn_counts(k)
            incorrect_synapses += pre_ct_k + post_ct_k
            mesh1 = vol.mesh.get(str(k))
            mesh1 = mesh1[int(k)]
            mesh_obj1 = trimesh.Trimesh(mesh1.vertices, mesh1.faces)
            mesh_total_inc += mesh_obj1

    
    prop_synap_correct = total_synapses / (total_synapses + incorrect_synapses)
    prop_vol_correct = mesh_total / (mesh_total + mesh_total_inc)

    return prop_synap_correct, prop_vol_correct


In [26]:
correct_extension = df.loc[df['Root_id'] == (str(864691136577830164) + '/')]
correct_extension.drop(correct_extension.index[correct_extension['Ext'] == 'null/'], inplace = True)
total_synapses = 0
all_extensions = correct_extension['Ext'].to_list()

total = []
for i in all_extensions:
    pre_ct, post_ct = get_syn_counts(i[:-1])
    total.append( pre_ct + post_ct)

non_correct = []

for j in [864691135826764827, 864691135458639120]:
    pre_ct, post_ct = get_syn_counts(j)
    non_correct.append( pre_ct + post_ct)

sum(total)/(sum(total)+sum(non_correct))

C:\Users\disc2\AppData\Local\Temp\ipykernel_24092\3834407941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correct_extension.drop(correct_extension.index[correct_extension['Ext'] == 'null/'], inplace = True)


0.66

# create function 


Function where:
Inputs are (root_id, [extensions])
Function should:
Calculate how many synapses added by the extensions are correct / incorrect by comparing to the ground truth table
Calculate how much volume is added by the extensions (again total / correct / incorrect)